# Data Cleaning with Pandas

In this notebook we'll go through a few basic data cleaning steps that should be performed on all new datasets where necessary.

We'll go through the process with both the `orders` and `orderlines` datasets. You can then practice these skills by cleaning the `products` dataset yourself

In [4]:
import pandas as pd

In [5]:
# orders.csv
url = "https://drive.google.com/file/d/1Vu0q91qZw6lqhIqbjoXYvYAQTmVHh6uZ/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orders = pd.read_csv(path)

# orderlines.csv
url = "https://drive.google.com/file/d/1FYhN_2AzTBFuWcfHaRuKcuCE6CWXsWtG/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orderlines = pd.read_csv(path)

Before we begin, let's create a copy of the `orders` and `orderlines` DataFrames. This way we are sure any of our changes won't affect the original DataFrames

In [6]:
orders_df = orders.copy()

In [7]:
orderlines_df = orderlines.copy()

One of the best ways to begin data cleaning is by exploring using `.info()`. This will tell us:
* The shape of the DataFrame
* The names of the columns
* If there are any missing values
* The datatypes of the columns

By exploring the missing values and correcting any incorrect datatypes, we often come across inconsistencies in our data.

Beyond this, we should also have a **check for any duplicate rows**.

Let's first deal with the duplicates, as it's nice and easy, then we'll explore what `.info()` has to tell us.

## 1.&nbsp; Duplicates
We can check for duplicates using the pandas [.duplicated()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html) method.

We can then delete these rows, if we wish, using [.drop_duplicates()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html)

In [8]:
# orders_df
orders_df.duplicated().sum()

0

In [9]:
# orderlines_df
orderlines_df.duplicated().sum()

0

We have no duplicate rows in either DataFrame. Easy, there is no problem to solve. Normally though, if there were some duplicates, we'd drop the extra rows.

# 2.&nbsp; `.info()`

In [ ]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226909 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      226909 non-null  int64  
 1   created_date  226909 non-null  object 
 2   total_paid    226904 non-null  float64
 3   state         226909 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 6.9+ MB


* `total_paid` has 5 missing values
* `created_date` should become datetime datatype

In [ ]:
orderlines_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB


* `date` should be a datetime datatype
* `unit_price` should be a float datatype

## 3.&nbsp; Missing values

### 3.1.&nbsp; Orders
* `total_paid` has 5 missing values

In [10]:
num_missing = orders_df['total_paid'].isna().sum()
total_rows = orders_df.shape[0]
percent_missing = (100*num_missing/total_rows)
print(f"5 missing values represents {percent_missing:.5f}% of the rows in our DataFrame")

5 missing values represents 0.00220% of the rows in our DataFrame


> A quick way to find out a percentage if you don't need to print out a sentence for yourself/students/colleagues is `.value_count(normalize=True)`

In [ ]:
orders_df['total_paid'].isna().value_counts(normalize=True)

,proportion
total_paid,
False,0.999978
True,0.000022


As there is such a tiny amount of missing values, we will simply delete these rows, as we have enough data without them.

In [ ]:
orders_df = orders_df.dropna(axis=0)

Should you have a significant number of missing values in the future, you have a choice:
+ you can impute the values
+ you can take the values from other DataFrames if they are redundantly stored
+ you can delete the rows or columns
+ or any number of other creative solutions

Please, always consider how much time you have on your project, and what impact your method of choice will have on your final assesment.

### 3.2.&nbsp; Orderlines
There are no missing values in `orderlines_df`

## 4.&nbsp; Datatypes

### 4.1.&nbsp; Orders
* `created_date` should become datetime datatype

In [11]:
orders_df["created_date"] = pd.to_datetime(orders_df["created_date"])

### 4.1.&nbsp; Orderlines
* `date` should be a datetime datatype
* `unit_price` should be a float datatype

#### 4.1.1.&nbsp; `date`

In [12]:
orderlines_df["date"] = pd.to_datetime(orderlines_df["date"])

#### 4.1.2.&nbsp;`unit_price`

In [13]:
orderlines_df["unit_price"] = pd.to_numeric(orderlines_df["unit_price"])

ValueError: Unable to parse string "1.137.99" at position 6

As you can see when we try to convert `unit_price` to a numerical datatype, we receive a `ValueError` telling us that pandas doesn't understand the number `1.137.99`. This is probably because numbers cannot have multiple decimal points. Let's see if there are any other numbers like this:

> `.` is a wildcard in regex, we need the `\` as an escape

In [ ]:
# Count the number of decimal points in the unit_price
orderlines_df['unit_price'].str.count("\.").value_counts()

,count
unit_price,
1,257814
2,36169


Looks like over 36000 rows in `orderlines` are affected by this problem. Let's work out how much that is as a percentage of our total data.

In [14]:
# Count the rows with more than one `.`
mult_decimal_rows = (orderlines_df['unit_price'].str.count("\.")>1).sum()

# Find the percentage of corrupted rows
percent_corrupted = (100 * mult_decimal_rows / orderlines_df.shape[0])
print(f"{percent_corrupted:.2f}% of the rows in our DataFrame have multiple decimal points in the unit_price")

12.30% of the rows in our DataFrame have multiple decimal points in the unit_price


This is a bit of a tricky decision as 12.3% is a significant amount of our data... and we might even end up losing a larger portion of our data than this too. For the moment we will delete the rows as we only have 2 weeks for this project and I'd like some quick, accurate results to show. If we have time at the end, we can come back and investigate this problem further, maybe there's a solution?

Each row of `orderlines` represents a product in an order. For example, if order number 175 contained 3 seperate products, then order 175 would have 3 rows in `orderlines`, one row for each of the products. If 2 of those products have 'normal' prices (14.99, 15.85) and 1 has a price with 2 decimal points (1.137.99), we need to remove the whole order and not just the affected row. If we only remove the row with 2 decimal places then any later analysis about products and prices could be misleading.

We therefore need to find the order numbers associated with the rows that have 2 decimal points, and then remove all the associated rows.

In [ ]:
# Boolean mask to find the orders that contain a price with multiple decimal points
multiple_decimal_mask = orderlines_df['unit_price'].str.count("\.") > 1

# Apply the boolean mask to the orderlines DataFrame. This way we can find the order_id of all the affected orders.
corrupted_order_ids = orderlines_df.loc[multiple_decimal_mask, "id_order"]

# Keep only the rows that do not have multiple decimal points
orderlines_df = orderlines_df.loc[~orderlines_df['id_order'].isin(corrupted_order_ids)]

In [ ]:
orderlines_df.shape[0]

216250

We still have 216250 rows in orderlines to work with. This should be more than enough for our evaluation.

Now that all of the 2 decimal point prices have been removed, let's try again to convert the column `unit_price` to the correct datatype.

In [ ]:
orderlines_df["unit_price"] = pd.to_numeric(orderlines_df["unit_price"])

It worked perfectly

# Challenge: Clean the `products` DataFrame
Now it's your turn. Use the lessons you learnt above and clean the products DataFrame. You don't have to copy exactly what we did. Think about the consequences of your actions, sometimes it is ok to delete rows, other times you may wish to come up with more creative solutions.

In [2]:
import pandas as pd

In [3]:
# products.csv
url = "https://drive.google.com/file/d/1afxwDXfl-7cQ_qLwyDitfcCx3u7WMvkU/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
products = pd.read_csv(path)

In [ ]:
# your code here

### Look for Duplicates

In [ ]:
# your code here

### Look for Missing values


In [ ]:
# your code here

### Check / Change Data types

In [ ]:
# your code here